In [1]:
!pip install -q transformers datasets evaluate sacrebleu accelerate bitsandbytes peft nltk pandas

import nltk
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('omw-1.4')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 30.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 39.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 113.0 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 86.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/

[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...


True

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from peft import PeftModel
import evaluate
import pandas as pd
from tqdm import tqdm
import os
import numpy as np

base_path = "/kaggle/input/vlsp-dataset" 

path_model_en_vi = "/kaggle/input/nllb-eng-to-vie/pytorch/default/1/export_model_eng_to_vie" 
path_model_vi_en = "/kaggle/input/nllb-vie-to-eng/pytorch/default/1/export_model_vie_to_eng"

base_checkpoint = "facebook/nllb-200-distilled-600M"
device = "cuda" if torch.cuda.is_available() else "cpu"

metric_bleu = evaluate.load("sacrebleu")
metric_ter = evaluate.load("ter")
metric_meteor = evaluate.load("meteor")

2025-12-14 17:55:03.157344: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765734903.337770      47 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765734903.392349      47 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /usr/share/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [3]:
def run_evaluation(adapter_path, src_lang_code, tgt_lang_code, test_file_src, test_file_tgt, output_name):
    print(f"\n{'='*10} REVIEWING: {src_lang_code} -> {tgt_lang_code} {'='*10}")
    
    print("1.Loading model...")
    tokenizer = AutoTokenizer.from_pretrained(base_checkpoint, src_lang=src_lang_code)
    base_model = AutoModelForSeq2SeqLM.from_pretrained(base_checkpoint)
    
    try:
        model = PeftModel.from_pretrained(base_model, adapter_path)
    except Exception as e:
        print(f"Error at {adapter_path}: {e}")
        return
        
    model.to(device)
    model.eval()
    
    print("2. Reading test...")
    with open(test_file_src, 'r', encoding='utf-8') as f:
        src_texts = f.read().splitlines()
    with open(test_file_tgt, 'r', encoding='utf-8') as f:
        ref_texts = f.read().splitlines()

    print(f"3. Translating... {len(src_texts)} câu...")
    predictions = []
    tgt_lang_id = tokenizer.convert_tokens_to_ids(tgt_lang_code)
    
    batch_size = 16
    for i in tqdm(range(0, len(src_texts), batch_size)):
        batch_src = src_texts[i : i + batch_size]
        
        inputs = tokenizer(batch_src, return_tensors="pt", padding=True, truncation=True, max_length=128).to(device)
        
        with torch.no_grad():
            generated_tokens = model.generate(
                **inputs,
                forced_bos_token_id=tgt_lang_id,
                max_length=128,
                num_beams=5, 
                early_stopping=True
            )
        
        decoded = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
        predictions.extend([d.strip() for d in decoded])

    print("4. Score...")
    bleu_refs = [[r] for r in ref_texts]
    
    score_bleu = metric_bleu.compute(predictions=predictions, references=bleu_refs)
    score_ter = metric_ter.compute(predictions=predictions, references=ref_texts)
    score_meteor = metric_meteor.compute(predictions=predictions, references=ref_texts)
    
    print(f"\n>>> RESULT: {src_lang_code}->{tgt_lang_code}:")
    print(f"   BLEU  : {score_bleu['score']:.2f}")
    print(f"   TER   : {score_ter['score']:.2f}")
    print(f"   METEOR: {score_meteor['meteor']:.4f}")
    
    print("5. Saving file...")
    df = pd.DataFrame({
        "Source": src_texts,
        "Reference": ref_texts,
        "Prediction": predictions
    })
    
    df["Sentence_BLEU"] = [metric_bleu.compute(predictions=[p], references=[[r]])['score'] for p, r in zip(predictions, ref_texts)]
    
    csv_filename = f"Analysis_{output_name}.csv"
    df.to_csv(csv_filename, index=False, encoding='utf-8-sig')
    print(f"   File saved: {csv_filename}")
    
    del model
    del base_model
    torch.cuda.empty_cache()
    print("-" * 40)

In [4]:
run_evaluation(
    adapter_path=path_model_en_vi,    
    src_lang_code="eng_Latn",
    tgt_lang_code="vie_Latn",
    test_file_src=os.path.join(base_path, "public_test.en.txt"),
    test_file_tgt=os.path.join(base_path, "public_test.vi.txt"),
    output_name="En_Vi"
)


========== REVIEWING: eng_Latn -> vie_Latn ==========
1.Loading model...


tokenizer_config.json:   0%|          | 0.00/564 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.3M [00:00<?, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

...

model.safetensors:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

2. Reading test...
3. Translating... 3000 câu...


100%|██████████| 188/188 [20:45<00:00,  6.62s/it]


4. Score...

>>> RESULT: eng_Latn->vie_Latn:
   BLEU  : 38.45
   TER   : 53.25
   METEOR: 0.6424
5. Saving file...
   File saved: Analysis_En_Vi.csv
----------------------------------------


In [5]:
run_evaluation(
    adapter_path=path_model_vi_en,    
    src_lang_code="vie_Latn",
    tgt_lang_code="eng_Latn",
    test_file_src=os.path.join(base_path, "public_test.vi.txt"), 
    test_file_tgt=os.path.join(base_path, "public_test.en.txt"), 
    output_name="Vi_En"
)


========== REVIEWING: vie_Latn -> eng_Latn ==========
1.Loading model...
2. Reading test...
3. Translating... 3000 câu...


100%|██████████| 188/188 [20:01<00:00,  6.39s/it]


4. Score...

>>> RESULT: vie_Latn->eng_Latn:
   BLEU  : 31.34
   TER   : 63.50
   METEOR: 0.5930
5. Saving file...
   File saved: Analysis_Vi_En.csv
----------------------------------------
